In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from citipy import citipy
import json
import requests
from random import uniform

In [ ]:
api_key = '223e00908491fdb261c8989de790f010'
location_data = pd.DataFrame()
location_data['ran_lat'] = [np.random.uniform(-90,90) for x in range(1350)]
location_data['ran_long'] = [np.random.uniform(-180, 180) for x in range(1350)]

location_data['closest_city'] = ""
location_data['country'] = ""

for index, row in location_data.iterrows():
    lat = row['ran_lat']
    lng = row['ran_long']
    location_data.set_value(index, 'closest_city', citipy.nearest_city(lat, lng).city_name)
    location_data.set_value(index, 'country', citipy.nearest_city(lat, lng).country_code)

    
location_data

In [ ]:
location_data = location_data.drop_duplicates(subset='closest_city').reset_index()

In [ ]:
location_data["Temp"] = ""
location_data["Humidity"] = ""
location_data["Wind Speed"] = ""
location_data["Cloudiness"] = ""
del location_data['index']
location_data.head()

In [ ]:
units = "Imperial"

counter = 0
for index, col in location_data.iterrows():
    target_url = "http://api.openweathermap.org/data/2.5/weather?units=%s&APPID=%s&q=%s" % (units, api_key, col['closest_city'])
    try:
        map_data = requests.get(target_url).json()
        location_data.set_value(index, "Temp", map_data["main"]["temp_max"])
        location_data.set_value(index, "Humidity", map_data["main"]["humidity"])
        location_data.set_value(index, "Wind Speed", map_data["wind"]["speed"])
        location_data.set_value(index, "Cloudiness", map_data["clouds"]["all"])
        print("------------------------")
        print("Getting data for the city:" , counter, ':' , map_data["name"])
    except:
        print("Missing Data for this city")
    counter = counter + 1

In [ ]:
location_data=location_data.replace('', np.NaN,regex=True)
location_data = location_data.dropna(axis=0, how='all',subset=['Temp'])
location_data.to_csv("latandlong.csv",encoding="utf-8", index=False)


In [ ]:
plt.scatter(location_data["ran_lat"], location_data["Temp"], marker="o", color = 'blue')

plt.title("Max Temp vs Latitude")
plt.ylabel("Max Temperature in F")
plt.xlabel("Latitude")
plt.yticks(np.arange(-60, 130, 20))
plt.grid(True)

plt.savefig("Latvstemp.png")

plt.show()

In [ ]:
plt.scatter(location_data["ran_lat"], location_data["Humidity"], marker="o", color = 'blue')

plt.title("Humidity vs Latitude")
plt.ylabel("% Humidity")
plt.xlabel("Latitude")
plt.grid(True)
plt.yticks(np.arange(-20, 130, 20))

plt.savefig("Latvshumidity.png")

plt.show()

In [ ]:
plt.scatter(location_data["ran_lat"], location_data["Cloudiness"], marker="o", color = 'blue')

plt.title("Cloudiness vs Latitude") 
plt.ylabel("% Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)
plt.yticks(np.arange(-20, 140, 20))

plt.savefig("Latvsclouds.png")

plt.show()


In [ ]:
plt.scatter(location_data["ran_lat"], location_data["Wind Speed"], marker="o", color = 'blue')

plt.title("Wind Speed vs Latitude")
plt.ylabel("Wind Speed in mph")
plt.xlabel("Latitude")
plt.yticks(np.arange(-5, 60, 10))
plt.grid(True)

plt.savefig("Latvswindspeed.png")

plt.show()